<a href="https://colab.research.google.com/github/tonyjosephsebastians/AI-Design-patterns/blob/main/GROUP_1_%E2%80%94_%E2%80%9CRequests_are_slow_or_unreliable%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔵 GROUP 1 — Requests Are Slow or Unreliable  
## Foundation Patterns (Start Here)

This group addresses the **most common and dangerous failure** in AI systems:

> **AI workloads are slow, flaky, and expensive — but beginners design APIs as if they are fast and reliable.**

We will intentionally build a **bad system first**, feel the pain, and then fix it step by step.

---

## 🎯 Patterns Covered in Group 1

We learn these **together** because they solve the *same class of problems*:

1. Sync vs Async Execution  
2. Long-Running Task Pattern  
3. Job / Workflow Pattern  
4. Retry + Backoff Pattern  
5. Timeout Pattern  
6. Circuit Breaker Pattern  
7. Partial Result Pattern  
8. Graceful Degradation Pattern  

---

# 🧪 STEP 1 — CREATE THE FAILURE  
**(DO THIS FIRST — DO NOT SKIP)**

---

## 🎯 Goal of Step 1

Understand **why synchronous APIs break** for AI workloads.

If you don’t *feel* this failure, the patterns will feel abstract.

---

## 🧱 What We Will Build (INTENTIONALLY BAD)

A naive API that:
- uploads a document
- does “heavy AI processing” (OCR, embeddings, etc.)
- **blocks the HTTP request**

⚠️ This is exactly how many beginner AI APIs are built.

---

## 📌 Colab Cell 1 — Install Dependencies

```bash
!pip install fastapi uvicorn nest_asyncio


In [1]:
!pip install fastapi uvicorn nest_asyncio

In [2]:
import time
import nest_asyncio
from fastapi import FastAPI
from fastapi.responses import JSONResponse

nest_asyncio.apply()

app = FastAPI()

@app.post("/documents")
def uploadDocument():
  time.sleep(20)
  return JSONResponse({"status":"done"})

In [ ]:
import uvicorn
import asyncio

config = uvicorn.Config(app, host="0.0.0.0", port=8000, loop="asyncio")
server = uvicorn.Server(config)
asyncio.run(server.serve())

#PATTERN #1 — Sync vs Async Execution

Anything that may take more than a few seconds must be asynchronous.

Why Sync APIs Fail for AI

Sync APIs assume:

fast execution

reliable downstream services

no retries

AI workloads are:

slow

flaky

retry-prone

rate-limited (429s)

❌ Sync + AI = broken system

Type of Work	API Style
Chat responses	Sync (often streaming)
OCR, embeddings, indexing	Async
Agent workflows	Async
Batch extraction	Async

Async by default for AI pipelines

STEP 2 — APPLY THE FIRST FIX

Introduce an Async Boundary
🎯 Goal of Step 2

Return immediately to the client

Move heavy work out of the request lifecycle

This introduces two patterns at once:

Sync vs Async Execution

Long-Running Task Pattern

In [9]:
import uuid
import time
from fastapi import BackgroundTasks

jobs = {}  # in-memory job store (temporary)

@app.post("/documents")
def uploadDocument(background_tasks: BackgroundTasks):
  jobId = str(uuid.uuid4())
  jobs[jobId] = "pending"
  background_tasks.add_task(processDocument, jobId)
  return JSONResponse({
        "job_id": job_id,
        "status": "queued"
    })


def process_document(job_id: str):
    jobs[job_id]["status"] = "running"
    time.sleep(20)  # simulate heavy AI work
    jobs[job_id]["status"] = "done"

In [8]:
@app.get("/jobs/{job_id}")
def get_job_status(job_id: str):
      return jobs.get(job_id, {"error": "job not found"})


PATTERNS YOU JUST LEARNED
✅ Pattern 1 — Sync vs Async Execution

API responds immediately

Work continues in background

✅ Pattern 2 — Long-Running Task Pattern

Heavy AI work detached from HTTP request

No blocking of workers

✅ Pattern 3 — Job / Workflow Pattern

Explicit job identifier

Client polls job status

Work is trackable

Current limitations:

❌ No failure handling

❌ No retries

❌ No timeout protection

❌ No circuit breaker

❌ No progress tracking

# 🔵 GROUP 1 — Step 3  
## Job States + Failure Handling  
**(Workflow Pattern + State Pattern — GoF)**

---

## 🧠 Why Step 3 exists (Failure First)

Right now, our async version stores only:

```python
jobs[job_id] = {"status": "queued"}


What breaks?

If background processing crashes, the job can get stuck forever

There is no progress visibility

Failures are not captured

No clear lifecycle guarantees

So we need a real job lifecycle (state machine).

e will:

Define explicit job states

Track progress

Capture errors

Enforce a simple, predictable lifecycle

Patterns learned here:

✅ Job / Workflow Pattern

✅ State Pattern (GoF)

✅ Partial Result (foundation via progress reporting)

Job State Machine

Production-friendly lifecycle:

queued → running → succeeded
              ↘ failed


(We can add cancelled later.)